## Материалы для знакомства c PyTorch и Transformers

* https://www.youtube.com/watch?v=qKL9hWQQQic
* https://arxiv.org/pdf/1706.03762.pdf
* https://towardsdatascience.com/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452
* https://huggingface.co/blog/bert-101

* https://huggingface.co/docs/transformers/main/en/index
* https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
* https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/


## Transformers

Transformers - это пакет, который предоставляет инструменты для простой загрузки и использования предобученных SOTA моделей машинного обучения. 

In [1]:
import json
from pprint import pprint
import re

import pandas as pd
import torch
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from transformers import AutoModel, AutoTokenizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
# специальный объект-токенизатор, который преобразует строки к нужному модели виду
rubert_tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

In [3]:
# создаем объект-модель
# предупреждение говорит о том, что эту модель надо дообучать
rubert_model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Создание и обучение модели

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin


class FullDescriptionCreator(BaseEstimator, TransformerMixin):
    """Добавляет столбец с полным описанием вакансии"""

    patt = re.compile("[^\s\w]")


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        return X

In [5]:
class BertEmbedder(BaseEstimator, TransformerMixin):
    """Получаете эмбеддинги для батча текстов"""

    def __init__(self, bert_tokenizer, bert_model):
        self.bert_tokenizer = bert_tokenizer
        self.bert_model = bert_model

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        with torch.no_grad():
            t = self.bert_tokenizer(
                X.tolist(), padding=True, truncation=True, return_tensors="pt"
            )

            model_output = self.bert_model(**t)
            embeddings = model_output.last_hidden_state[:, 0, :]
            embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.numpy()

In [19]:
# загрузка данных (обсуждали на прошлой встрече)
# оставим от трейна 3 тыс. строчек, потому что все 15 тыс. обработать сразу с бертом
# требует слишком много вычислительных ресурсов
train = pd.read_csv("C:/Users/denis/Desktop/hackaton/train.csv", index_col="index")
targets = train['target'].value_counts().index
train = train.query("target != -1")

while True:
    k = 1
    for i in targets: 
        if (train[train['target']==i].shape[0]) == 1:
            train.drop(train[train['target'] == i].sample(1).index[0], axis=0, inplace=True)
            k = 2   
    if k == 1:
        break
train['name'] = train['name'].str.replace(r"\([^()]*\)", " ", regex=True).str.replace(r'[^\w\s]+', ' ',regex=True)

X_train, y_train = train.drop(columns=['target']), train["target"]

In [20]:
train['target'].value_counts().tail(10)

,name,description,target
index,,,
169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
169293782,продавец консультант yota,<p>За любыми достижениями нашей компании в пер...,5223
291073919,электросварщик накс ск,<p>Группе компаний КСК на промышленный объект ...,7212
179260831,кассир,Правила приема и проведения расчетных и конвер...,5223
39608878,оператор машинного доения,"Выполнять все виды работ, связанные с производ...",6121
...,...,...,...
110663394,офис менеджер,<p><strong>Обязанности:</strong></p> <p>- испо...,3341
30812168,офис менеджер,<strong>Обязанности:</strong> <ul> <li>Прием и...,3341
50002587,офис менеджер,<p><strong>Обязанности:</strong></p> <ul> <li>...,3341


In [21]:
#parameters_grid = {
 #      'alpha' : np.linspace(0.00001, 0.0001, 3),
  #     'learning_rate': ['optimal', 'constant', 'invscaling'],
   #    'eta0' : np.linspace(0.00001, 0.0001, 3),
    #   'max_iter' : np.arange(50,450,100),
  # }
#classifier = SGDClassifier(random_state = 0, tol=1e-3)
#cv = model_selection.StratifiedShuffleSplit(n_splits=2, test_size = 0.2)
#grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', verbose=10)

In [30]:
%%time
clf_bert = Pipeline(
    [
        (
            "bert",
            # изменен способ векторизации текстов
            ColumnTransformer([("vectorize", BertEmbedder(rubert_tokenizer, rubert_model), "name")]),
        ),
        (
            "clf",
            # заменена модель-классификатор
            MLPClassifier(alpha=0.0001, max_iter=500)
            ,
        ),
    ]
)

clf_bert.fit(X_train, y_train)
print(clf_bert.score(X_train, y_train))

0.9845080340567185
CPU times: total: 2min 30s
Wall time: 1min 10s


In [31]:
test = pd.read_csv("C:/Users/denis/Desktop/hackaton/test.csv")


test['name'] = test['name'].str.replace(r"\([^()]*\)", " ", regex=True).str.replace(r'[^\w\s]+', ' ',regex=True)


y_pred = clf_bert.predict(test)

submission = test[['index']].assign(target=y_pred)
display(submission.head(40))
submission.to_csv("C:/Users/denis/Desktop/otvet.csv", index=False)

,index,target
0,26461447,8322
1,26464220,4323
2,26467473,5223
3,26468989,1324
4,26471705,4222
5,26476036,5412
6,26480630,9621
7,26480759,7212
8,26483824,8322
9,26483941,4222
